# Image Captioning with an Encoder-Decoder RNN

This notebook demonstrates how to build an image captioning model using a classic encoder-decoder architecture. This is a fascinating multi-modal task that combines Computer Vision (to understand the image) and Natural Language Processing (to generate a descriptive sentence).

### **📌 The Core Idea: Encoder-Decoder**
The model consists of two main parts that work together:

1.  **The Encoder**: A pre-trained Convolutional Neural Network (CNN), like ResNet, acts as the encoder. It takes an image as input and outputs a compact feature vector (an embedding). This vector is a rich, numerical representation of the image's content—its "thought" vector.

2.  **The Decoder**: A Recurrent Neural Network (RNN), specifically an LSTM in our case, acts as the decoder. It takes the image's feature vector from the encoder and generates the caption one word at a time. It learns the statistical structure of language to produce coherent sentences.

In [ ]:
from IPython.display import display

In [ ]:
# %pip install torch torchvision pillow spacy numpy
# %pip install torchtext
# %pip install pycocotools

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import random
from tqdm import tqdm
import kagglehub
import pandas as pd
from PIL import Image
import spacy
from IPython.display import display
import pickle
import os
import matplotlib.pyplot as plt

## 1️⃣ The Dataset: Flickr8k
We will use the Flickr8k dataset, a popular benchmark for image captioning. It contains:
- **8,000 Images**: Sourced from the Flickr website.
- **40,000 Captions**: Each image is paired with five different human-generated captions, providing a rich variety of descriptions.

First, we'll download the dataset from Kaggle Hub and inspect its contents.

In [ ]:
# Download the Flickr8k dataset
path = kagglehub.dataset_download("adityajn105/flickr8k")
print("Path to dataset files:", path)

In [ ]:
# Load the captions from the `captions.txt` file
captions_path = os.path.join(path, "captions.txt")
captions_df = pd.read_csv(captions_path)
print(f"Captions loaded: {len(captions_df)} entries")
print(captions_df.head())

In [ ]:
# Images directory
images_dir = os.path.join(path, "Images")
print(f"Images directory: {images_dir}")
print(f"Number of images: {len(os.listdir(images_dir))}")

In [ ]:
# Let's look at a sample image and one of its captions
print("Sample caption:", captions_df.iloc[0]['caption'])
sample_image_path = os.path.join(images_dir, captions_df.iloc[0]['image'])
sample_img = Image.open(sample_image_path)
display(sample_img)

## 2️⃣ Building the Vocabulary

Neural networks work with numbers, not words. Therefore, we need to build a **vocabulary** to map every unique word in our dataset to a unique integer index. We also add special tokens that are crucial for the training process:

- `<PAD>`: A padding token used to make all sequences in a batch the same length.
- `<SOS>`: A "Start of Sentence" token that signals the beginning of a caption to the decoder.
- `<EOS>`: An "End of Sentence" token that the model learns to predict when it has finished generating a caption.
- `<UNK>`: An "Unknown" token for words that appear in the test set but not in our training vocabulary.

We use the `spaCy` library for robust and efficient tokenization.

In [ ]:
spacy_eng = spacy.load("en_core_web_sm") # you might need to run this: python -m spacy download en_core_web_sm

In [ ]:
def word_tokenize(text):
    return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

In [ ]:
# Initialize vocabulary with special tokens
word_to_index = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
index_to_word = {idx: word for word, idx in word_to_index.items()}
word_freq = {}

# Build vocabulary from all captions in the dataset
for caption in tqdm(captions_df['caption']):
    tokens = word_tokenize(caption.lower())
    
    for token in tokens:
        if token not in word_to_index:
            idx = len(word_to_index)
            word_to_index[token] = idx
            index_to_word[idx] = token
            word_freq[token] = 1
        else:
            word_freq[token] += 1

print(f"\nVocabulary size: {len(word_to_index)}")

In [ ]:
word_tokenize('<SOS> hi, my friend <EOS>')  # We will manually add tokens for <EOS> and <SOS> etc after tokenization to avoid them breaking up.

## 3️⃣ The Model: A CNN-RNN Encoder-Decoder
Our model is composed of two main modules: an Encoder to process the image and a Decoder to generate the text.

### 🔹 The Encoder (CNN)
We use a pre-trained ResNet-50 model, which has already learned rich feature representations from the large ImageNet dataset. We perform two key modifications:
1.  **Freeze Layers**: We freeze all the convolutional layers (`requires_grad_(False)`) to prevent them from being updated during training. This is a form of transfer learning that speeds up training and prevents overfitting.
2.  **Replace Classifier**: We chop off the final, fully-connected classification layer (which originally predicts 1000 ImageNet classes) and replace it with a new `nn.Linear` layer. This new layer will output a feature vector of `embed_size`, which will be the input to our decoder.

### 🔹 The Decoder (RNN)
The decoder is an LSTM (Long Short-Term Memory) network, a type of RNN well-suited for sequence generation.

1.  **Embedding Layer**: It first converts the integer-coded words of the caption into dense vectors of size `embed_size`.
2.  **Input Concatenation**: The decoder's first input is the image feature vector from the encoder. For subsequent steps, it takes the embedded representation of the previously generated word.
3.  **LSTM**: The LSTM processes the sequence of inputs, maintaining a hidden state that keeps track of the context of the generated sentence.
4.  **Linear Layer**: A final linear layer maps the LSTM's output to the size of our vocabulary, producing a probability distribution over all possible next words.

### 🔹 The Inference Method (`caption_image`)
This method is used for generating new captions after the model is trained. It works *autoregressively*:
1.  The image is passed through the encoder to get its feature vector.
2.  This feature vector is fed into the decoder to predict the first word (which should be `<SOS>`).
3.  The predicted word is then fed back into the decoder as input for the next time step.
4.  This process repeats until the model predicts the `<EOS>` token or a maximum length is reached.

## 4️⃣ The PyTorch Dataset and DataLoader
We create a custom `Flickr8kDataset` class to interface with our data. This is the standard PyTorch way to prepare data for a `DataLoader`.

The `__getitem__` method is the most important part. For a given index `idx`, it performs these steps:
1.  Loads the image and its corresponding caption from our dataframe.
2.  Applies the necessary image transformations (resizing to a fixed size, converting to a tensor, and normalizing).
3.  Tokenizes the caption, converts it to a sequence of indices, and adds the `<SOS>` and `<EOS>` tokens.
4.  Pads the caption tensor with the `<PAD>` token index so that all sequences in a batch have the same length (`max_seq_len`). This is required for batch processing.

## 5️⃣ Model Training
Now we define the hyperparameters, initialize the model, and set up the optimizer and loss function for training.

### 🔹 Hyperparameters & Setup
These are the key settings for our model and training process. Tuning these can significantly impact performance.

### 🔹 Loss Function and Optimizer
- **Loss Function**: We use `nn.CrossEntropyLoss`. This is the standard choice for multi-class classification, which is what we're doing at each time step—predicting the next word from the entire vocabulary.
Importantly, we set `ignore_index=word_to_index['<PAD>']`. This tells the loss function to ignore the `<PAD>` tokens when calculating the loss, so the model isn't penalized for its predictions on padded parts of the sequence.

- **Optimizer**: We use the Adam optimizer, a popular and effective choice for deep learning models.

### 🔹 Pre-training Check
Before we start training, let's see what our randomly initialized model produces. We expect the captions to be nonsensical.

### 🔹 The Training Loop
The training process involves iterating through the dataset for a number of epochs. In each step:

1.  **Teacher Forcing**: We feed the ground-truth caption (shifted by one position) into the decoder at each time step, regardless of the decoder's own prediction from the previous step. This is a technique called *teacher forcing*, which stabilizes training and helps the model learn the language structure more quickly. We feed `captions[:, :-1]` (from `<SOS>` to the second-to-last word) and expect the model to predict `captions[:, 1:]` (from the first word to `<EOS>`).
2.  **Calculate Loss**: The model's output (logits) is compared against the target sequence to calculate the loss.
3.  **Backpropagation**: The loss is backpropagated through the network to compute gradients.
4.  **Gradient Clipping**: We clip the gradients (`clip_grad_norm_`) to a maximum value. This is a common practice in RNN training to prevent the "exploding gradients" problem.
5.  **Optimizer Step**: The optimizer updates the model's trainable parameters (the decoder and the final layer of the encoder).

### 🔹 Plotting the Training Progress
Visualizing the loss helps us understand how well the model is learning. A decreasing loss indicates that the model is successfully minimizing its prediction error over time.

## 6️⃣ Saving and Loading the Model

To avoid retraining the model every time we want to use it, we can save its state. It's good practice to save three things:
1.  **Model State Dictionary**: The learned weights and biases of the model.
2.  **Vocabulary Mappings**: The `word_to_index` and `index_to_word` dictionaries are essential for converting between words and indices during inference.
3.  **Model Configuration**: The hyperparameters used to build the model (`embed_size`, `hidden_size`, etc.), so we can perfectly reconstruct the architecture before loading the weights.

## 7️⃣ Generating Captions (Inference)
Finally, let's see our trained model in action! We will use the same test images from our pre-training check and compare the new, hopefully much-improved, captions.

The model is set to evaluation mode (`model.eval()`) to disable layers like Dropout that are only used during training.

## 8️⃣ Conclusion & Next Steps

We have successfully built, trained, and evaluated an image captioning model using a CNN encoder and an RNN decoder. While the results after a short training run are promising, they can be significantly improved.

### **📝 Exercises for Further Exploration**
1.  **More Training**: The most impactful change would be to train for more `epochs` on a larger portion of the dataset (or the full dataset). Deep learning models are data-hungry!
2.  **Hyperparameter Tuning**: Experiment with different values for `embed_size`, `hidden_size`, `num_decoder_layers`, and `learning_rate`.
3.  **Different RNNs**: Replace the `nn.LSTM` with a `nn.GRU` in the decoder. Compare the results and training time.
4.  **Attention Mechanism**: The biggest architectural improvement for this task is to add an **attention mechanism**. Attention allows the decoder to dynamically focus on different parts of the image as it generates each word, often leading to more accurate and contextually relevant captions. This is a key component in modern sequence-to-sequence models.
5.  **Beam Search Decoding**: The current `caption_image` method uses greedy decoding (picking the single best word at each step). Implement **beam search**, which keeps track of the `k` most probable sequences at each step, often resulting in higher-quality final captions.

### Contributed by: Ali Habibullah(Changed from Telha Bilal original notebook 🫡)